In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
#from tensorflow.keras.layers import LSTM
from tensorflow.keras import Sequential,layers,callbacks
from tensorflow.keras.layers import Dense,LSTM,Dropout,GRU,Bidirectional
from tensorflow.keras.models import Model
import pandas as pd
import math
import keras
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import save_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Sequential
#from kerastuner.tuners import RandomSearch
#from kerastuner.engine.hyperparameters import HyperParameters
#from pickle import dump,load
#import warnings


import wandb
from wandb.integration.keras import WandbCallback
wandb.login()





#Step 1: Read and explore data
file='C:/Users/Addisu/Desktop\kidi files/Python workshop with ellen/MeteharaPR.csv'
raw_data = pd.read_csv(file, parse_dates = ['DATE'],
                       index_col = 'DATE')
df = raw_data.copy()
# Outlier detection
up_b = df['PCP'].mean() + 2*df['PCP'].std()
low_b = df['PCP'].mean() - 2*df['PCP'].std()


# Replace outlier by interpolation for base consumption
df.loc[df['PCP'] > up_b, 'PCP'] = np.nan
df.loc[df['PCP'] < low_b, 'PCP'] = np.nan
df['PCP'].interpolate(inplace=True)

# Split train data and test data
train_size = int(len(df)*0.9)
train_dataset, test_dataset = df.iloc[:train_size],df.iloc[train_size:]

#Split the target variable and dependent variables
# Split train data to X and y
X_train = train_dataset.drop('PCP', axis = 1)
y_train = train_dataset.loc[:,['PCP']]
# Split test data to X and y
X_test = test_dataset.drop('PCP', axis = 1)
y_test = test_dataset.loc[:,['PCP']]

# Different scaler for input and output
scaler_x = MinMaxScaler(feature_range = (0,1))
scaler_y = MinMaxScaler(feature_range = (0,1))
# Fit the scaler using available training data
input_scaler = scaler_x.fit(X_train)
output_scaler = scaler_y.fit(y_train)
# Apply the scaler to training data
train_y_norm = output_scaler.transform(y_train)
train_x_norm = input_scaler.transform(X_train)
# Apply the scaler to test data
test_y_norm = output_scaler.transform(y_test)
test_x_norm = input_scaler.transform(X_test)

# Create a 3D input
def create_dataset (X, y, time_steps = 1):
    Xs, ys = [], []
    for i in range(len(X)-time_steps):
        v = X[i:i+time_steps, :]
        Xs.append(v)
        ys.append(y[i+time_steps])
    return np.array(Xs), np.array(ys)
TIME_STEPS = 30
X_test, y_test = create_dataset(test_x_norm, test_y_norm,   
                                TIME_STEPS)
X_train, y_train = create_dataset(train_x_norm, train_y_norm, 
                                  TIME_STEPS)
print('X_train.shape: ', X_test.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_train.shape)

#def create_model_LSTM(units):
    #model = Sequential()
    #model.add(LSTM (units = units, return_sequences = True,input_shape = [X_train.shape[1], X_train.shape[2]]))
    #model.add(Dropout(0.2))
    #model.add(LSTM (units = units))
    #model.add(Dropout(0.2))
    #model.add(Dense(units = 1))
    #Compile model
    #model.compile(loss='mse', optimizer='adam')
    #return model
# Create BiLSTM model
#def create_model_bilstm(units):
    #model = Sequential()
    #model.add(Bidirectional(LSTM(units = units,                             
              #return_sequences=True),
              #input_shape=(X_train.shape[1], X_train.shape[2])))
    #model.add(Bidirectional(LSTM(units = units)))
    #model.add(Dense(1))
    #Compile model
    #model.compile(loss='mse', optimizer='adam')
    #return model
# Create LSTM or GRU model
def create_model_GRU(units):
    model = Sequential()
    model.add(LSTM(units = units, return_sequences = True,input_shape = [X_train.shape[1], X_train.shape[2]]))
    model.add(Dropout(0.1))
    model.add(LSTM(units = units))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    #Compile model
    model.compile(loss='mse', optimizer='adam')
    return model

# GRU and LSTM
model = create_model_GRU(64)
#model = create_model_lstm(64)

#model = create_model_bilstm(32)
model.summary()

def get_optimizer(lr=1e-3, optimizer="adam"):
    "Select optmizer between adam and sgd with momentum"
    if optimizer.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=lr)
    if optimizer.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.1)
   

def train(model, batch_size=64, epochs=10, lr=1e-3, optimizer='adam', log_freq=10):  
    
    # Compile model like you usually do.
    tf.keras.backend.clear_session()
    model.compile(loss="mse", 
                  optimizer=get_optimizer(lr, optimizer))
    # earrly-stopper
    #early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # callback setup
    cbs = [WandbCallback(data_type='time series', log_batch_frequency=log_freq)]

    model.fit(X_train, 
              y_train, 
              batch_size=batch_size, 
              epochs=epochs, 
              validation_data=(X_test, y_test), 
              callbacks=cbs)

sweep_config = {
    'method': 'bayes'
    }

metric = {
    'name': 'val_loss',
    'goal': 'minimize' 
    }

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'dropout': {
          'values': [0.1,0.2, 0.3]
        },
    'dense_activation':{
           'values': ['relu', 'sigmoid']
    }}

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'value': 1}
    })

import math

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.01
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 128,
      }
    })

import pprint

pprint.pprint(sweep_config)

def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project nam
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration, if any
        wandb.config.architecture_name = "LSTM"
        wandb.config.dataset_name = "pcpm"

        # initialize model
        model =create_model_GRU(wandb.config.batch_size)

        train(model, 
              wandb.config.batch_size, 
              wandb.config.epochs,
              wandb.config.learning_rate,
              wandb.config.optimizer)

sweep_id = wandb.sweep(sweep_config, project="sweeps-keras")

wandb.agent(sweep_id, function=sweep_train, count=50)

X_train.shape:  (847, 30, 11)
y_train.shape:  (7859, 1)
X_test.shape:  (847, 30, 11)
y_test.shape:  (7859, 1)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            19456     
                                                                 
 dropout (Dropout)           (None, 30, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 52545 (205.25 KB)
Trainable params: 52545 (205.25 KB)
Non-traina

wandb: Agent Starting Run: ed8zwr9a with config:
wandb: 	batch_size: 72
wandb: 	dense_activation: relu
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	learning_rate: 0.008989038957605503
wandb: 	optimizer: adam


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING The data_type argument of wandb.keras.WandbCallback is deprecated and will be removed in a future release. Please use input_type instead.
wandb: WARNING Setting input_type = data_type.


110/110 [==============================] - ETA: 0s - loss: 0.0535

C:\anaconda3\envs\jupyterenv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

197/197 [==============================] - ETA: 0s - loss: 0.0269

C:\anaconda3\envs\jupyterenv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: C:\Users\Addisu\Desktop\kidi files\Python workshop with ellen\wandb\run-20240621_151623-qy3x40om\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\Addisu\Desktop\kidi files\Python workshop with ellen\wandb\run-20240621_151623-qy3x40om\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\Addisu\Desktop\kidi files\Python workshop with ellen\wandb\run-20240621_151623-qy3x40om\files\model-best)... Done. 0.0s


197/197 [==============================] - 22s 91ms/step - loss: 0.0269 - val_loss: 0.0206
